In [43]:
import config
import ssl
import nltk
nltk.download('stopwords')


from pymongo import MongoClient

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer


# Import linear kernel
from sklearn.metrics.pairwise import linear_kernel




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dehkharghanielnaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
mongo_url = config.MONGO_URL
client = MongoClient(mongo_url, ssl_cert_reqs=ssl.CERT_NONE)
db = client['vendors_data_db']

# Three different collections (1. Users 2. Products 3. Bookmarks)
collection_users = db.users_col
collection_product = db.products
collection_bookmarks = db.bookmarks_col

In [45]:
# changing the collections to Dataframes
users_df = pd.DataFrame(list(collection_users.find()))
products_df = pd.DataFrame(list(collection_product.find()))
bookmarks_df = pd.DataFrame(list(collection_bookmarks.find()))

In [46]:
# User dataframe
users_df.head(1)

,_id,firstName,lastName,gender,phone,email,password,countryOfOrigin,dateOfBirth
0,6133af8bc4a7e1daa5fd78e0,Lisa,Roach,Female,700.110.0724x387,Lisa.Roach@gmail.com,$2b$10$UXmmmXeFmsH36HwkwEEsFeCgTAdMlrcY2o4PeNK...,Georgia,1995-04-26


In [47]:
# Bookmark dataframe
bookmarks_df.head(1)

,_id,userId,active,created,productId,modified
0,6135b4cbf810533516c4639f,6133af8dc4a7e1daa5fd7923,True,2021-09-06 08:27:23,6132464b75e744d165bf2b08,None


In [48]:
# Products dataframe
products_df.head(1)

,_id,title,brand,price,quantity,discount,image,vendor,unitPrice,oldPrice,unitPriceQuantity,offerDuration,category,subcategory,badges,thumbnailImgs,description,properties,ingredients,preparationInstruction,hints,manufacturer,nutritionalValues,status,created,modified
0,613515cdae5592de9045691b,Feines Gemüse-Allerlei 190g,BEBIVITA,0.65,190g Glas,0.0,https://media.kaufland.com/images/PPIM/AP_Prod...,Kaufland,0.65,0.0,"100 g = 0,35 €",None,Baby & Kind,Baby- & Kindernahrung (350),None,[https://media.kaufland.com/images/PPIM/AP_Pro...,"Beste Zutaten aus der Natur - Karotten, Kartof...",Altersempfehlung:ab 6. MonatFüllmenge:190.0Fre...,"Gemüse 73% (Karotten, Kartoffeln, Erbsen), Was...",,Bei Flüssigkeitsabsatz bitte umrühren. Benötig...,Bebivita GmbH,None,True,1630594239201,2021-09-03 18:00:51.834


In [49]:
users_df.columns

Index(['_id', 'firstName', 'lastName', 'gender', 'phone', 'email', 'password',
       'countryOfOrigin', 'dateOfBirth'],
      dtype='object')

In [50]:
products_df.columns

Index(['_id', 'title', 'brand', 'price', 'quantity', 'discount', 'image',
       'vendor', 'unitPrice', 'oldPrice', 'unitPriceQuantity', 'offerDuration',
       'category', 'subcategory', 'badges', 'thumbnailImgs', 'description',
       'properties', 'ingredients', 'preparationInstruction', 'hints',
       'manufacturer', 'nutritionalValues', 'status', 'created', 'modified'],
      dtype='object')

In [51]:
bookmarks_df.columns

Index(['_id', 'userId', 'active', 'created', 'productId', 'modified'], dtype='object')

# Data Cleaning

In [52]:
# Dropping unnessary columns in users
users_info = users_df.drop(columns= ['firstName', 'lastName', 'phone' ,'email', 'password'])

In [53]:
# Renaming in user's df
users_info = users_info.rename(index=str, columns= {'_id':'user_id'})
users_info.head(1)

,user_id,gender,countryOfOrigin,dateOfBirth
0,6133af8bc4a7e1daa5fd78e0,Female,Georgia,1995-04-26


In [54]:
# Dropping unnessary columns in products
products_info = products_df.drop(columns= ['image', 'thumbnailImgs', 'status'])


In [55]:
# Renaming in product's df

products_info = products_info.rename(index=str, columns= {'_id':'product_id'})
products_info.head(1)

,product_id,title,brand,price,quantity,discount,vendor,unitPrice,oldPrice,unitPriceQuantity,offerDuration,category,subcategory,badges,description,properties,ingredients,preparationInstruction,hints,manufacturer,nutritionalValues,created,modified
0,613515cdae5592de9045691b,Feines Gemüse-Allerlei 190g,BEBIVITA,0.65,190g Glas,0.0,Kaufland,0.65,0.0,"100 g = 0,35 €",None,Baby & Kind,Baby- & Kindernahrung (350),None,"Beste Zutaten aus der Natur - Karotten, Kartof...",Altersempfehlung:ab 6. MonatFüllmenge:190.0Fre...,"Gemüse 73% (Karotten, Kartoffeln, Erbsen), Was...",,Bei Flüssigkeitsabsatz bitte umrühren. Benötig...,Bebivita GmbH,None,1630594239201,2021-09-03 18:00:51.834


In [56]:
# Dropping unnessary columns in bookmarks
bookmarks_info = bookmarks_df.drop(columns= ['_id','created', 'modified','active'])

In [57]:
# Renaming in bookmark's df
bookmarks_info = bookmarks_info.rename(index=str, columns= {'_id':'bookmarks_id', 'userId': 'user_id', 'productId' : 'product_id'})
bookmarks_info.head(3)

,user_id,product_id
0,6133af8dc4a7e1daa5fd7923,6132464b75e744d165bf2b08
1,6133af8dc4a7e1daa5fd7928,6132465275e744d165bf2ca1
2,6133af8ec4a7e1daa5fd794b,6132466275e744d165bf2faa


# Machine Learning:

In [58]:

products_info.head(20)['ingredients']

0     Gemüse 73% (Karotten, Kartoffeln, Erbsen), Was...
1     MILCHZUBEREITUNG (MILCH* 50%, Wasser, ENTRAHMT...
2     Gemüse* 77% (Karotten*, Tomaten*, Kartoffeln* ...
3     Birnenpüree* 64%, Bananenpüree* 28%, Orangensa...
4     Gemüse* 75% (Karotten, Kartoffeln, Erbsen, Gem...
5     Wasser, Gemüse* 37,7% (Karotten 20%, Pastinake...
6     Karotten* 40%, Wasser, Reis* gekocht 13,2%, (W...
7     WEIZENMEHL 65,5%, Zucker, Palmöl, GERSTENMALZE...
8     Früchte* 59% (säurearmer Apfelsaft* aus Apfels...
9     Gemüse* 45% (Kartoffeln* 19%, Karotten*, Kürbi...
10    Gemüse* 57% (Tomaten *28%, Karotten*, Pastinak...
11    Gemüse* 44% (Tomaten*, Karotten*, SELLERIE*, Z...
12                  Äpfel*. *aus biologischer Erzeugung
13    Früchte* 50% (Pfirsiche* 18%, Aprikosen* 12%, ...
14    Gemüse*55% (Kartoffeln* 38%, Karotten* 10%, La...
15    Zutaten: Gemüse* 67% (Kartoffeln* 32%, Karotte...
16    Zutaten: Gefriergetrocknete Mangostücke* (100%...
17    Zutaten: Pürierte Banane (43%), pürierter 

In [59]:
# Stopwords for German Language
german_stop_words = stopwords.words('german')

# Define a TF-IDF vectorizer object. Remove all german stop words.
tfidf = TfidfVectorizer(stop_words= german_stop_words)

# Replay NaN with an empty string
products_info['ingredients'] = products_info['ingredients'].fillna('')

# Construct the required TF-IDF matrix by fitting and transforming data
tfidf_matrix = tfidf.fit_transform(products_info['ingredients'])

# Output the shape of tfidf_matrix
tfidf_matrix.shape



(3967, 5264)

In [60]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [61]:
# Constract a reverse map of indices and product titles
indices = pd.Series(products_info.index, index=products_info['title']).drop_duplicates()
print(indices.head(5))

title
Feines Gemüse-Allerlei 190g                               0
ohne Zuckerzusatz Haferbrei pur 190g                      1
Kartoffeln mit mildem Rahm-Gulasch mit Bio-Rindfleisch    2
100% Birne, Banane, Orange & Vanille 100g                 3
Kleine Entdecker Buntes Buttergemüse 190g                 4
dtype: object


In [66]:
# Function that takes in product title as input and outputs most similar product
def get_recommendations(title, cosine_sim = cosine_sim):
    # Get the index of the product that matches the title
    idx = indices[title]
    print([idx])

    # Get the pairwise similarity scores of all products with that product
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    #print(sim_scores)

    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key= lambda x:x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:3]

    # Get the movie indices 
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return products_info['title'].iloc[product_indices]


In [67]:
 # Get bookmarks from user1 prod1, prd2, prod3
 # fro prod in producs_bookmarks get_recommendations(products-title)
get_recommendations('Folgemilch 3 ab dem 10. Monat 500g')

['136']


76    Gemüsereis mit Bio-Hühnchen 190g
77    Gemüse-Reis mit Rindfleisch 250g
Name: title, dtype: object